In [1]:
# from pymongo import MongoClient
import pandas as pd
import requests
from pprint import pprint

In [4]:
# client = MongoClient()
# db = client['meetup']

In [2]:
API_KEY = '6cc211017f64141072497d1766550'
BASE_URL = 'https://api.meetup.com'

In [3]:
# get a listing of all meetup events hosted by DE-ProD-SF

def get_events(urlname):
    
    events_url = BASE_URL + '/' + urlname + '/events'
    params = {'sign':'true','key': API_KEY, 'status': 'past'}
    response = requests.get(events_url, params = params)
    events = response.json()
    
    return events

urlname = 'DE-ProD-SF'
get_events(urlname)

[{u'created': 1455863910000,
  u'description': u"<p>Exercise your critical thinking with a data eng challenge problem! Hear how others would solve the problem, what trade-offs they consider, and voice your opinion / ask questions in a non-threatening atmosphere. Like a journal club, but minus reading boring papers, and plus an exciting problem :-)</p> <p>I'll send out the challenge problem by email to the RSVP'd group 2 days before the event.</p> <p>By the way, employers rank whiteboarding &amp; critical thinking skills as one of their top priorities in hiring data engineers. Practice makes perfect!</p> <p>See you soon,<br/>Aaron</p> ",
  u'group': {u'created': 1441761756000,
   u'id': 18913207,
   u'join_mode': u'open',
   u'lat': 37.779998779296875,
   u'lon': -122.41999816894531,
   u'name': u'Data Engineering Professional Development San Francisco',
   u'urlname': u'DE-ProD-SF',
   u'who': u'Professional Developers'},
  u'id': u'228968660',
  u'link': u'http://www.meetup.com/DE-Pro

In [57]:
#query all event ids and descriptions

def query_events():
    
    event_id = []
    event_description= []
    event_time = []
    for event in get_events(urlname):
        event_id.append(event['id'])
        event_description.append(event['description'])
        event_time.append(event['time'])
 
    return event_id, event_description, event_time

# events_ids = [''.join(event['id']) for event in events]
# print events_ids

# events_descriptions = [event['description'] for event in events]
# print events_descriptions

event_id, event_description, event_time = query_events()

df_events= pd.DataFrame([event_id, event_description, event_time]).T
df_events.columns = ['event_id', 'event_description', 'event_time']
df_events

,event_id,event_description,event_time
0,228968660,<p>Exercise your critical thinking with a data...,1456455600000
1,228648803,"<p><b>*Note</b>, expedite your check in at Gal...",1459301400000


In [56]:
#get all rsvps for each event

member_id = []
event_id = []

for i in event_id:
    rsvps_url = BASE_URL + '/' + urlname +'/events/' + i + '/rsvps'
    params = {'sign':'true','key': API_KEY}
    response = requests.get(rsvps_url, params = params)
    rsvps = response.json()
    
    for rsvp in rsvps:
        member_id.append(rsvp['member']['id'])
        event_id.append(rsvp['event']['id'])
               
df_rsvps = pd.DataFrame([member_id, event_id]).T
df_rsvps.columns = ['member_id', 'event_id']
df_rsvps['rsvp'] = 1
df_rsvps

,member_id,event_id,rsvp
0,87429312,228968660,1
1,149624662,228968660,1
2,129647262,228968660,1
3,132426572,228968660,1
4,67449572,228968660,1
5,111657512,228968660,1
6,198433955,228968660,1
7,11847383,228968660,1
8,10048695,228968660,1
9,14704591,228968660,1


In [66]:
# # join df_events and df_rsvps
# df = pd.merge(df_events, df_rsvps, on = 'event_id')
# df['rsvp'] = 1
# df

In [58]:
# get all members from a group (how to bypass the 200 limit?)
members_url = BASE_URL + '/2/members' 
params = {'sign':'true','key': API_KEY, 'group_id': '18913207'}
response = requests.get(members_url, params = params)
members_all = response.json()
pprint (members_all)

{u'meta': {u'count': 149,
           u'description': u'API method for accessing members of Meetup Groups',
           u'id': u'',
           u'lat': u'',
           u'link': u'https://api.meetup.com/2/members',
           u'lon': u'',
           u'method': u'Members',
           u'next': u'',
           u'signed_url': u'https://api.meetup.com/2/members?offset=0&format=json&group_id=18913207&page=200&order=name&sig_id=192752466&sig=80e5454b9c1f5954af7d129eef86c3981cd1916f',
           u'title': u'Meetup Members v2',
           u'total_count': 149,
           u'updated': 1461800733000,
           u'url': u'https://api.meetup.com/2/members?offset=0&sign=True&format=json&group_id=18913207&page=200&key=6cc211017f64141072497d1766550&order=name'},
 u'results': [{u'city': u'San Francisco',
               u'country': u'us',
               u'id': 194699547,
               u'joined': 1459391446000,
               u'lat': 37.79,
               u'link': u'http://www.meetup.com/members/194699547',
 

In [7]:
members = members_all['results']
member_id = [member['id'] for member in members]
join_time = [member['joined'] for member in members]

df_members = pd.DataFrame([member_id,join_time]).T
df_members.columns = ['member_id', 'join_time']
df_members

,member_id,join_time
0,194699547,1459391446000
1,87429312,1441761756000
2,81401992,1459377236000
3,185745656,1456036929000
4,11885543,1459228941000
5,55930862,1445727497000
6,195886534,1459373753000
7,192607432,1442034315000
8,183554416,1459380534000
9,183993413,1450205662000


In [59]:
# np.sort(df_members.member_id.astype(int))

array([   292159,    621496,   1209999,   2040157,   2963626,   3651127,
         3881071,   3893698,   5057169,   5057483,   5251626,   6804434,
         6887205,   7880057,   8028690,   8155925,   8575630,   9730843,
        10048695,  10134330,  10591017,  10609056,  11298443,  11847383,
        11885543,  12324218,  12968024,  13908753,  14553888,  14704591,
        18222031,  18937971,  22424691,  25502362,  30445632,  35030162,
        40798102,  41888302,  44789882,  50412492,  51737362,  54092012,
        55930862,  57468772,  64180502,  65386392,  67449572,  75723792,
        77380882,  80950832,  81401992,  83275192,  85929712,  87429312,
        89525712,  89894632,  93671472,  97686732,  97988222,  99543232,
       101970742, 103328952, 105121292, 105218652, 105678462, 105796662,
       106924652, 111657512, 112225022, 116324022, 117627052, 119785422,
       120284782, 125516362, 127593662, 129647262, 130484162, 131591372,
       132426572, 133272042, 134357542, 137748422, 

In [19]:
df_members['key'] = 2
df_events['key'] = 2
df_1 = pd.merge(df_members, df_events, on = 'key')
df_1.drop('key', axis = 1, inplace = True)
df_1


,member_id,join_time,event_id,event_description,event_time
0,194699547,1459391446000,228968660,<p>Exercise your critical thinking with a data...,1456455600000
1,194699547,1459391446000,228648803,"<p><b>*Note</b>, expedite your check in at Gal...",1459301400000
2,87429312,1441761756000,228968660,<p>Exercise your critical thinking with a data...,1456455600000
3,87429312,1441761756000,228648803,"<p><b>*Note</b>, expedite your check in at Gal...",1459301400000
4,81401992,1459377236000,228968660,<p>Exercise your critical thinking with a data...,1456455600000
5,81401992,1459377236000,228648803,"<p><b>*Note</b>, expedite your check in at Gal...",1459301400000
6,185745656,1456036929000,228968660,<p>Exercise your critical thinking with a data...,1456455600000
7,185745656,1456036929000,228648803,"<p><b>*Note</b>, expedite your check in at Gal...",1459301400000
8,11885543,1459228941000,228968660,<p>Exercise your critical thinking with a data...,1456455600000
9,11885543,1459228941000,228648803,"<p><b>*Note</b>, expedite your check in at Gal...",1459301400000


In [60]:
df_2 = df_1[(df_1['join_time'] < df_1['event_time'])]
df_2
df_2.drop(['join_time','event_time'], axis = 1, inplace = True)
df_2

/Users/datascientist/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,member_id,event_id,event_description
2,87429312,228968660,<p>Exercise your critical thinking with a data...
3,87429312,228648803,"<p><b>*Note</b>, expedite your check in at Gal..."
6,185745656,228968660,<p>Exercise your critical thinking with a data...
7,185745656,228648803,"<p><b>*Note</b>, expedite your check in at Gal..."
9,11885543,228648803,"<p><b>*Note</b>, expedite your check in at Gal..."
10,55930862,228968660,<p>Exercise your critical thinking with a data...
11,55930862,228648803,"<p><b>*Note</b>, expedite your check in at Gal..."
14,192607432,228968660,<p>Exercise your critical thinking with a data...
15,192607432,228648803,"<p><b>*Note</b>, expedite your check in at Gal..."
18,183993413,228968660,<p>Exercise your critical thinking with a data...


In [64]:
df_3 = pd.merge(df_2, df_rsvps, how ='left', on = ['member_id','event_id'])
df_3.fillna(value = 0)

,member_id,event_id,event_description,rsvp
0,87429312,228968660,<p>Exercise your critical thinking with a data...,1
1,87429312,228648803,"<p><b>*Note</b>, expedite your check in at Gal...",1
2,185745656,228968660,<p>Exercise your critical thinking with a data...,0
3,185745656,228648803,"<p><b>*Note</b>, expedite your check in at Gal...",0
4,11885543,228648803,"<p><b>*Note</b>, expedite your check in at Gal...",0
5,55930862,228968660,<p>Exercise your critical thinking with a data...,0
6,55930862,228648803,"<p><b>*Note</b>, expedite your check in at Gal...",0
7,192607432,228968660,<p>Exercise your critical thinking with a data...,0
8,192607432,228648803,"<p><b>*Note</b>, expedite your check in at Gal...",0
9,183993413,228968660,<p>Exercise your critical thinking with a data...,0


In [69]:
# df_2['rsvp_2'] = ((df_2['rsvp'].notnull()) | (df_2['join_time'] < df_2['event_time'])).astype(int)
# df_2['rsvp_2'] = ((df_2['rsvp'].notnull()) | (df_2['join_time'] < event_time)).astype(int)
# df_2['rsvp_2'] = ((df_2.rsvp == 0) & (df_2.join_time < df_2.event_time)).astype(int)

In [34]:
# final df
# df = pd.DataFrame([member_id, event_id, event_description, rsvp])
# df.columns = ['member_id', 'event_id', 'event_description', 'rsvp']